In [1]:
import math
import numpy as np
import pandas as pd



In [2]:
df_fin = pd.read_csv('./pickle_files/df_final_spark_top5k_080421.csv',index_col=0)

In [3]:
df_fin_true = df_fin[df_fin['has_enough_stat']]

In [4]:
df_fin_true.head()


,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey
0,1000012530,0.016220,99.99,2021-02-13,66.081387,-0.0187,0.0059,True,44.680,247523697279378
2,1000021922,0.023490,114.99,2021-02-16,90.113095,-0.0194,0.0025,True,NaN,156995084198890
3,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,11954970507506
5,1000034743,0.005481,279.99,2021-02-04,110.202494,-0.0186,0.0048,True,171.030,206871238538442
10,1000145006,0.004873,289.99,2021-02-04,188.190476,-0.0086,0.0025,True,188.819,280637889289282


In [5]:
df_final_spark=df_fin_true.astype({'pc2': 'float64'})

print(df_final_spark.dtypes)

df_final_spark.dropna(inplace=True)
df_final_spark = df_final_spark[df_final_spark['PE']< 1.0]
df_final_spark = df_final_spark[df_final_spark['PE_err']<1.0]

item_code             int64
avg_conv_rate       float64
last_price          float64
last_date            object
avg_unique_views    float64
PE                  float64
PE_err              float64
has_enough_stat        bool
pc2                 float64
item_skey             int64
dtype: object


In [6]:
VAT = 1.19



In [23]:
df_n_views = pd.read_csv('./../raw_data/Excel_files/Top5k_View_PE.csv',index_col = 0)

In [24]:
df_n_views.head()

,View_PE,View_PE_err
1000012530,-1.6265,0.4565
1000021922,-1.0147,0.1548
1000034762,-1.3782,0.3497
1000125896,-1.6598,0.3366
1000005727,-0.9853,0.2722


In [25]:
df_n_views['item_code'] = df_n_views.index

In [26]:
df_n_views.head()

,View_PE,View_PE_err,item_code
1000012530,-1.6265,0.4565,1000012530
1000021922,-1.0147,0.1548,1000021922
1000034762,-1.3782,0.3497,1000034762
1000125896,-1.6598,0.3366,1000125896
1000005727,-0.9853,0.2722,1000005727


In [29]:
df_merged = pd.merge(df_final_spark,df_n_views, on="item_code")

In [30]:
df_merged.head()


,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,View_PE,View_PE_err
0,1000012530,0.016220,99.99,2021-02-13,66.081387,-0.0187,0.0059,True,44.680,247523697279378,-1.6265,0.4565
1,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,11954970507506,-2.0624,0.3918
2,1000145006,0.004873,289.99,2021-02-04,188.190476,-0.0086,0.0025,True,188.819,280637889289282,-2.7277,0.5322
3,1000005727,0.017079,99.99,2021-02-25,170.099034,-0.0246,0.0044,True,40.729,71488185050378,-0.9853,0.2722
4,1000016028,0.016290,139.99,2021-02-25,26.446154,-0.0328,0.0057,True,52.430,48800168481821,-1.6602,0.2891


In [96]:
def create_auxiliary_var(dataf=df_merged):
    dataf['Delta_price'] = (dataf['last_price']/VAT - dataf['pc2'])
    dataf['Margin_base'] = dataf['avg_unique_views']*dataf['avg_conv_rate']*(dataf['Delta_price'])

def classify(dataf= df_merged):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_up'] - 3*dataf['Margin_err_up']), 'up',
                                        np.where(dataf['Margin_base'] < (dataf['Margin_down'] - 3*dataf['Margin_err_down']), 'down','neutral'))

def classify_up(dataf=df_merged):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_up'] - 3*dataf['Margin_err_up']), 'up','neutral_up')

def classify_down(dataf=df_merged):
    dataf['Group_flag'] = np.where(dataf['Margin_base'] < (dataf['Margin_down'] - 3*dataf['Margin_err_down']), 'down','neutral_down')

In [97]:
def create_Margin( increment = 0.01, dataf = df_merged, flag = 'up'):
    if flag == 'down':
        increment = -increment

    price          = 'price_'+flag
    Delta_price    = 'Delta_price_'+flag
    conv_rate      = 'conv_rate_'+flag
    conv_rate_err  = 'conv_rate_err_'+flag
    n_views        = 'avg_unique_views_' + flag
    n_views_err    = 'avg_unique_views_err_' + flag
    Margin         = 'Margin_'+flag
    Margin_err     = 'Margin_err_'+flag

    #CROSS CHECK THE WHOLE ERROR PROPAGATION!!!!

    dataf[price]              =  dataf['last_price']*math.exp(increment)
    dataf[Delta_price]        =  (dataf[price]/VAT - dataf['pc2'])
    dataf[conv_rate]          =  dataf['avg_conv_rate']*np.exp(dataf['PE']*np.log(dataf[price]/dataf['last_price']))
    dataf[conv_rate_err]      =  np.fabs(dataf[conv_rate])*np.fabs(dataf['PE_err']*np.log(dataf[price]/dataf['last_price']))
    dataf[n_views]            =  dataf['avg_unique_views']*np.exp(dataf['View_PE']*np.log(dataf[price]/dataf['last_price']))
    dataf[n_views_err]        =  np.fabs(dataf[n_views])*np.fabs(dataf['View_PE_err']*np.log(dataf[price]/dataf['last_price']))
    dataf[Margin]             =  dataf[n_views]*dataf[conv_rate]*(dataf[Delta_price])
    #Propagate Margin error
    dataf[Margin_err]         =  np.sqrt(np.power(dataf[conv_rate]*dataf[n_views_err]*(dataf[Delta_price]),2) + np.power(dataf[n_views]*dataf[conv_rate_err]*(dataf[Delta_price]),2))



In [98]:
#print(df_final_spark[['item_code','avg_conv_rate','last_price']].head())
#create_Margin(0.01)

In [104]:
original_df= df_merged
print(len(original_df.index))
tmp_df = pd.DataFrame()

original_df_up   = pd.DataFrame()
original_df_down = pd.DataFrame()

1000


In [105]:
list_of_columns_up   = ['item_code','last_date','last_price','price_up','PE','PE_err','View_PE','View_PE_err','Delta_price','pc2','avg_conv_rate','avg_unique_views','avg_unique_views_up','Margin_base','Margin_up','Group_flag']
list_of_columns_down = ['item_code','last_date','last_price','price_down','PE','PE_err','View_PE','View_PE_err','Delta_price','pc2','avg_conv_rate','avg_unique_views','avg_unique_views_down','Margin_base','Margin_down','Group_flag']
create_auxiliary_var(original_df)
for inc in [0.01,0.2,0.3,0.4]:#,0.3,0.4,0.5]: #np.linspace(0.01,0.1,10):
    print('Increment = ' + str(inc))
    if inc == 0.01:
        create_Margin(inc,original_df,'up')
        create_Margin(inc,original_df,'down')
        classify(original_df)
        tmp_df           = tmp_df.append(original_df[original_df['Group_flag'] == 'neutral'])
        original_df_up   = original_df_up.append(original_df[original_df['Group_flag'] == 'up'])
        original_df_up   = original_df_up[list_of_columns_up]
        original_df_down = original_df_down.append(original_df[original_df['Group_flag'] == 'down'])
        original_df_down   = original_df_down[list_of_columns_down]
        #print('tmp\n',tmp_df[list_of_columns].head())
        #print('up\n', original_df_up[list_of_columns].head())
        #print('down\n',original_df_down[list_of_columns].head())

    if inc > 0.01:
        create_Margin(inc,original_df_up,'up')
        create_Margin(inc,original_df_down,'down')
        classify_up(original_df_up)
        classify_down(original_df_down)
        tmp_df           = tmp_df.append(original_df_up[original_df_up['Group_flag'] == 'neutral_up'])
        tmp_df           = tmp_df.append(original_df_down[original_df_down['Group_flag'] == 'neutral_down'])
        original_df_up   = original_df_up[original_df_up['Group_flag'] == 'up']
        original_df_down = original_df_down[original_df_down['Group_flag'] == 'down']
        #print('tmp\n',tmp_df[list_of_columns].head())
        #print('up\n', original_df_up[list_of_columns].head())
        #print('down\n',original_df_down[list_of_columns].head())
    original_df_down['Margin_base']   = original_df_down['Margin_down']
    original_df_up['Margin_base']   = original_df_up['Margin_up']

    print(len(tmp_df.index))
    print(len(original_df_up.index))
    print(len(original_df_down.index))

Increment = 0.01
554
442
4
Increment = 0.2
733
267
0
Increment = 0.3
995
5
0
Increment = 0.4
998
2
0


In [107]:
tmp_df
#print(or_filt_df[or_filt_df['Group_flag'] =='down'])
#print(or_filt_df[or_filt_df['Group_flag'] =='up'])
#print(or_filt_df[or_filt_df['Group_flag'] =='neutral'])

,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,...,Margin_err_up,price_down,Delta_price_down,conv_rate_down,conv_rate_err_down,avg_unique_views_down,avg_unique_views_err_down,Margin_down,Margin_err_down,Group_flag
0,1000012530,0.016220,99.99,2021-02-13,66.081387,-0.0187,0.0059,True,44.680,2.475237e+14,...,0.193448,98.995083,38.509145,0.016223,9.571313e-07,67.164990,0.306608,41.959118,0.191559,neutral
1,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,1.195497e+13,...,0.145824,475.214020,122.469512,0.003122,1.030184e-07,95.202078,0.373002,36.397796,0.142612,neutral
4,1000016028,0.016290,139.99,2021-02-25,26.446154,-0.0328,0.0057,True,52.430,4.880017e+13,...,0.081313,138.597076,64.038131,0.016295,9.288084e-07,26.888878,0.077736,28.058380,0.081133,neutral
5,1000050787,0.000001,369.99,2021-02-20,27.333333,-0.0284,0.0087,True,150.389,5.032731e+13,...,0.000028,366.308538,157.433301,0.000001,8.702471e-11,28.151030,0.181630,0.004433,0.000029,neutral
8,1000176734,0.007736,159.99,2021-02-08,100.555556,-0.0149,0.0030,True,54.119,3.825599e+13,...,0.192924,158.398073,78.988624,0.007737,2.321088e-07,101.820389,0.313098,62.225675,0.191353,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,1000194221,0.025424,159.99,2021-01-08,40.125730,-0.0219,0.0041,NaN,103.349,NaN,...,5.076588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up
994,1000109909,0.012308,239.99,2021-02-25,55.833333,-0.0197,0.0023,NaN,76.090,NaN,...,5.182360,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up
7,1000175476,0.055748,83.99,2021-02-25,80.722222,-0.0505,0.0061,NaN,48.799,NaN,...,12.744743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up
301,1000149111,0.014237,139.99,2020-12-12,51.169643,-0.0224,0.0024,NaN,87.599,NaN,...,2.785603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral_up


In [113]:
tmp_df.set_index(['Group_flag','item_code']).count(level='Group_flag')

,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,View_PE,...,Margin_up,Margin_err_up,price_down,Delta_price_down,conv_rate_down,conv_rate_err_down,avg_unique_views_down,avg_unique_views_err_down,Margin_down,Margin_err_down
Group_flag,,,,,,,,,,,,,,,,,,,,,
neutral,554,554,554,554,554,554,554,554,554,554,...,554,554,554,554,554,554,554,554,554,554
neutral_down,4,4,4,4,4,4,0,4,0,4,...,0,0,4,4,4,4,4,4,4,4
neutral_up,440,440,440,440,440,440,0,440,0,440,...,440,440,0,0,0,0,0,0,0,0


In [53]:
df_final_spark.head(400)


,item_code,avg_conv_rate,last_price,last_date,avg_unique_views,PE,PE_err,has_enough_stat,pc2,item_skey,...,conv_rate_err_up,Margin_up,Margin_err_up,price_down,Delta_price_down,conv_rate_down,conv_rate_err_down,Margin_down,Margin_err_down,Group_flag
0,1000012530,0.016220,99.99,2021-02-13,66.081387,-0.0187,0.0059,True,44.680,247523697279378,...,9.567734e-07,43.067609,0.002541,98.995083,38.509145,0.016223,9.571313e-07,41.282173,0.002436,up
1,1000012794,0.014916,259.99,2021-02-25,41.525974,-0.0120,0.0052,False,141.050,151936474932722,...,7.755479e-07,49.314333,0.002564,257.403056,75.255089,0.014918,7.757341e-07,46.619245,0.002424,up
3,1000022179,0.003121,479.99,2021-02-06,93.258739,-0.0136,0.0033,True,276.870,11954970507506,...,1.029903e-07,37.993099,0.001254,475.214020,122.469512,0.003122,1.030184e-07,35.654815,0.001177,up
4,1000024320,0.024532,119.99,2020-12-01,22.354839,-0.0220,0.0097,False,65.109,31827848594342,...,2.379083e-06,20.142108,0.001954,118.796080,34.719638,0.024537,2.380130e-06,19.044767,0.001847,up
5,1000034743,0.005481,279.99,2021-02-04,110.202494,-0.0186,0.0048,True,171.030,206871238538442,...,2.630578e-07,40.235416,0.001931,277.204053,61.914582,0.005482,2.631557e-07,37.407263,0.001796,up
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,1000092348,0.019247,82.99,2021-02-11,59.380952,-0.0528,0.0131,True,47.710,187933000919936,...,2.520051e-06,25.965224,0.003401,82.164236,21.335576,0.019257,2.522714e-06,24.397661,0.003196,up
398,1000094568,0.002279,599.99,2021-02-25,126.133999,0.0010,0.0042,False,422.300,225901869982735,...,9.570737e-08,24.994836,0.001050,594.020000,76.876470,0.002279,9.570545e-08,22.095966,0.000928,up
399,1000110390,0.015830,94.99,2021-02-25,109.591837,-0.0309,0.0079,True,45.590,52261297703691,...,1.250158e-06,60.761396,0.004800,94.044834,33.439272,0.015835,1.250931e-06,58.028493,0.004584,up
400,1000116167,0.004657,949.99,2021-02-25,38.758131,-0.0092,0.0065,False,448.740,113806289644918,...,3.026897e-07,64.541263,0.004195,940.537442,341.627598,0.004658,3.027454e-07,61.670858,0.004009,up


In [54]:
tmp_df[['item_code','avg_conv_rate','last_price','PE','PE_err','has_enough_stat','pc2','price_up','price_down','Margin_base','Margin_up','Margin_err_up','Margin_down','Margin_err_down','Group_flag']].head(400)


,item_code,avg_conv_rate,last_price,PE,PE_err,has_enough_stat,pc2,price_up,price_down,Margin_base,Margin_up,Margin_err_up,Margin_down,Margin_err_down,Group_flag
2602,1000227560,0.001835,779.99,-0.5212,0.3953,False,404.139,2.120233e+03,772.228970,20.680345,6.593642e+01,2.606467e+01,19.834838,0.078407,neutral
2677,1000188224,0.020552,519.99,-0.4047,0.2807,False,338.740,1.413479e+03,514.816013,52.363032,2.902266e+02,8.146661e+01,48.292832,0.135558,neutral
21,1000022565,0.023989,149.99,-0.1456,0.1268,False,111.680,3.303750e+06,148.497575,123.407684,4.001202e+05,5.073525e+05,8.114193,0.010289,neutral
34,1000168702,0.052631,51.99,0.0846,0.2011,False,37.100,1.145156e+06,51.472691,156.680561,3.953508e+06,7.950505e+06,10.841371,0.021802,neutral
40,1000185455,0.023306,129.99,0.0175,0.0981,False,39.430,2.863220e+06,128.696578,221.742663,2.425326e+06,2.379245e+06,58.138645,0.057034,neutral
88,1000215684,0.030163,63.99,-0.0397,0.0399,False,38.780,1.409474e+06,63.353289,365.940566,2.823366e+06,1.126523e+06,51.282442,0.020462,neutral
141,1000149709,0.011639,259.99,-0.1033,0.0498,False,169.770,5.726661e+06,257.403056,130.651275,5.850636e+05,2.913617e+05,15.911799,0.007924,neutral
229,1000116111,0.007981,1399.99,0.0398,0.0402,False,649.210,3.083683e+07,1386.059867,643.969500,9.367312e+06,3.765659e+06,125.126297,0.050301,neutral
423,1000145703,0.195797,69.99,-0.2521,0.0660,True,57.000,1.541632e+06,69.293588,1448.025190,1.885844e+06,1.244657e+06,22.331300,0.014739,neutral
453,1000189535,0.015505,379.99,-0.0457,0.0336,False,246.650,8.369837e+06,376.209036,197.877197,1.484530e+06,4.988021e+05,23.176099,0.007787,neutral


In [ ]:
tmp_df = tmp_df.sort_index()

In [ ]:
tmp_df['Margin_last'] = tmp_df['Margin_base']
tmp_df.head()

In [ ]:
create_Margin(0.01,df_final_spark)
df_final_spark.head()

In [ ]:
tmp_df['Margin_base']=df_final_spark['Margin_base']

tmp_df['Margin_rate_to_base'] = np.log(tmp_df['Margin_down']/tmp_df['Margin_base'])
#tmp_df['Margin_rate_to_last'] = np.log(tmp_df['Margin_down']/tmp_df['Margin_last'])

In [ ]:
tmp_df[['item_code','avg_conv_rate'	,'last_price','avg_unique_views', 'price_down', 'Delta_price', 'Margin_down','Margin_base','Margin_down_err','Margin_last','Group_flag','Margin_rate_to_base']]

In [ ]:
print(tmp_df['Margin_rate_to_base'].mean()*100)

In [ ]:
tmp_df.to_csv('../raw_data/Excel_files/Margin_result_PE_analysis_PC2_no_stock_no_stat_filter_20210226.csv')